# Data Analysis with Python: Zero to Pandas - Course Project

This is my course project submission for the course [Data Analysis with Python: Zero to Pandas](https://jovian.ml/learn/data-analysis-with-python-zero-to-pandas).

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First, we'll begin by saving the file.

In [ ]:
!pip install jovian --upgrade --quiet
import jovian

In [ ]:
jovian.commit(project="zerotopandas-course-project")

# Selecting Data
Here we'll be loading the dataset (from [Kaggle](https://www.kaggle.com/shivamb/netflix-shows)). This dataset consists of TV shows and movies available on Netflix as of 2019, collected from Flixable (third-party Netflix search engine).

In [ ]:
#Setup additional libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
#Path of the file to read
netflix_filepath = "/kaggle/input/netflix-shows/netflix_titles.csv"

#Read the file into a dataframe
netflix_raw = pd.read_csv(netflix_filepath, index_col='show_id')

# Data Preparation and Cleaning
Here we'll be loading the dataset (from Kaggle) using Pandas and preparing the data for further analysis.

In [ ]:
netflix_raw.head()

In [ ]:
netflix_raw.info()

There are a few columns with some null values added in them. Some of those columns I don't intend to use, so I'll just drop them. As for the others, I will deal with them at the time of analysis.

In [ ]:
netflix_data = netflix_raw.drop(columns=['cast', 'country'])

I'll first add a couple new columns to separate out the month and year of the added dates.

In [ ]:
from datetime import datetime

netflix_data['month_added'] = pd.to_datetime(netflix_data['date_added']).dt.strftime('%B')

netflix_data['year_added'] = netflix_data['date_added'].str[-4:]
netflix_data['year_added'] = pd.to_numeric(netflix_data['year_added'], errors='coerce')

I'll also parse the `date_added` column into a DateTime format.

In [ ]:
netflix_data['date_added'] = pd.to_datetime(netflix_data['date_added'])
netflix_data['date_added']

I'll create separate dataframes for each type of content in Netflix.

In [ ]:
movie_data = netflix_data[netflix_data['type']=='Movie']
show_data = netflix_data[netflix_data['type']=='TV Show']

In [ ]:
#Change the format of duration in shows
show_data['duration'] = show_data['duration'].str.replace(' Season', '')
show_data['duration'] = show_data['duration'].str.replace('s', '')
show_data['duration'] = show_data['duration'].astype(int)

In [ ]:
#Change the format of duration in movies
movie_data['duration'] = movie_data['duration'].str.replace(' min', '')
movie_data['duration'] = movie_data['duration'].astype(int)

# Perform Exploratory Analysis & Visualization

Before delving into the data, let's briefly go over the history of Netflix for more context (sourced from [its wikipedia page](https://en.wikipedia.org/wiki/Netflix)).

## An Extremely Brief History of Netflix

Netflix was founded in 1997, but it wasn't until 2007 that it expanded to the online streaming service that it's widely known for today. In 2010, the streaming giant expanded to an international market by offering its service in Canada. Since 2012, Netflix started to take a more active approach to creating and distributing its own content.

## Analysis

Now, let's finally look at the data and do some basic analysis.

In [ ]:
#Setup style
sns.set_style('darkgrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['figure.facecolor'] = '#00000000'

## Ratio of Netflix Content
Looking at the data, we can calculate the ratio of movies to TV shows on Netflix:

In [ ]:
ax = sns.countplot(x='type', data=netflix_data);

plt.xlabel('')
plt.ylabel('Count')
plt.title("Amount of Movies vs. TV Shows in Netflix");

In [ ]:
movie_amt = movie_data.shape[0]
show_amt = show_data.shape[0]

print("There are {} movies in Netflix, or {:.2%} of the content.".format(movie_amt, movie_amt/(movie_amt+show_amt)))
print("There are {} TV shows in Netflix, or {:.2%} of the content.".format(show_amt, show_amt/(movie_amt+show_amt)))

From both the graph and the above math, roughly 2/3 of the content on Netflix are movies with the remaining for TV shows.

## Growth in Netflix content over the years

In [ ]:
movie_yearly = movie_data.copy()
show_yearly = show_data.copy()

In [ ]:
movie_yearly = movie_yearly['year_added'].value_counts().reset_index()
movie_yearly = movie_yearly.rename(columns = {'year_added': 'count', 'index' : 'year_added'})

show_yearly = show_yearly['year_added'].value_counts().reset_index()
show_yearly = show_yearly.rename(columns = {'year_added': 'count', 'index' : 'year_added'})

Since there were no TV shows added from 2009-2011, I'll add those years manually.

In [ ]:

new_row = [{'year_added': 2009, 'count': 0},
           {'year_added': 2010, 'count': 0},
           {'year_added': 2011, 'count': 0}
          ]

for row in new_row:
    show_yearly = show_yearly.append(row, ignore_index=True)

In [ ]:
#Reformat the years into int
show_yearly['year_added'] = show_yearly['year_added'].astype(int)
movie_yearly['year_added'] = movie_yearly['year_added'].astype(int)

Although  the data is up thru the end of 2019, there seems to be some content added in 2020. I'll delete those.

In [ ]:
movie_yearly = movie_yearly.where(movie_yearly['year_added']!=2020)
movie_yearly = movie_yearly.dropna()

In [ ]:
show_yearly = show_yearly.where(show_yearly['year_added']!=2020)
show_yearly = show_yearly.dropna()

In [ ]:
#Sort by years
movie_yearly = movie_yearly.sort_values('year_added')
show_yearly = show_yearly.sort_values('year_added')

#Redo the indices for organization
movie_yearly.reset_index(drop=True, inplace=True)
show_yearly.reset_index(drop=True, inplace=True)

In [ ]:
#Percentage of content
movie_yearly['percent'] = movie_yearly['count'].apply(lambda x: 100*x/sum(movie_yearly['count']))
show_yearly['percent'] = show_yearly['count'].apply(lambda x: 100*x/sum(show_yearly['count']))

In [ ]:
plt.bar(show_yearly['year_added'], show_yearly['count'], color='r')
plt.bar(movie_yearly['year_added'], movie_yearly['count'], bottom=show_yearly['count']);

plt.title("Netflix Content added over the Years");

Netflix is increasing its content of both movies and TV shows annually. But it's a bit hard to tell whether they're both increased at the same rates.

In [ ]:
show_yearly['growth'] = 0

for year in range(11):
    if(show_yearly.iloc[year, 1] != 0):
        show_yearly.iloc[year+1, 3] = show_yearly.iloc[year+1, 1]/show_yearly.iloc[year, 1]
    
show_yearly.iloc[0, 3] = np.nan

In [ ]:
movie_yearly['growth'] = 0

for year in range(11):
    if(movie_yearly.iloc[year, 1] != 0):
        movie_yearly.iloc[year+1, 3] = movie_yearly.iloc[year+1, 1]/movie_yearly.iloc[year, 1]
        
movie_yearly.iloc[0, 3] = np.nan

In [ ]:
sns.lineplot(x='year_added', y='growth', data=show_yearly, label='TV Shows')
sns.lineplot(x='year_added', y='growth', data=movie_yearly, label='Movies')

plt.legend()
plt.xlabel("Year")
plt.ylabel("Growth as % of Prior Year")
plt.title("Content Growth in Netflix Annually");

Looking at the growth rate of added content, there doesn't seem to be any particular trend or stable increase. But we can at least see that Netflix drastically increased its movie catalog in 2011.

## Content Length

How long is most of Netflix's content?

In [ ]:
sns.distplot(a=movie_data['duration'])

plt.xlabel("Duration (mins)")
plt.title("Distribution of Movie Duration");

From the above histogram, we can see a majority of the Netflix movies are roughly 100 minutes long.

In [ ]:
sns.countplot(data=show_data, x='duration')

plt.title('Length of Netflix TV Shows')
plt.xlabel('# of Seasons');

Meanwhile, an extremely large portion of the TV Shows in Netflix are only one season long.

In [ ]:
jovian.commit(project="zerotopandas-course-project", privacy='private')

# Ask & Answer Questions about the Data
I'll ask 4 interesting questions about the dataset and answer them.

## *Q1: What type of TV show should a producer make to be featured in Netflix?*

I'll answer this by looking at the most popular TV shows in Netflix (indicated by the number of seasons the show has). From there, we can look at the genre it's listed under to see what type of show a producer should make.

In [ ]:
shows_genres = show_data.copy()

#Turn the genres of listed_in into a list to enable one hot encoding
shows_genres['listed_in'] = shows_genres['listed_in'].str.split(', ')
shows_genres['listed_in']

In [ ]:
#Iterate through the list of genres and place a 1 in the corresponding column for every row in the dataframe
for index, row in shows_genres.iterrows():
    for genre in row['listed_in']:
        shows_genres.loc[index, genre] = 1

#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
shows_genres = shows_genres.fillna(0)

In [ ]:
shows_genres.columns[11:33]

The genres list shows something strange... It lists `TV Shows` as a genre. 

In [ ]:
shows_genres[shows_genres['TV Shows']==1]

With random quick googling, some seem to be miscategorized content (Metallica: Some Kind of Monster and 7 (Seven) are movies). Since there aren't a lot of the `TV Shows` genre, I'll just drop them.

In [ ]:
shows_genres = shows_genres.drop(index=shows_genres[shows_genres['TV Shows']==1].index)
shows_genres = shows_genres.drop(columns='TV Shows')

In [ ]:
#Reformat the years into int from float
shows_genres['year_added'] = shows_genres['year_added'].astype(int)

In [ ]:
#Sort the top 10 shows with the most seasons
shows_top10 = shows_genres.sort_values('duration', ascending=False).head(10)

In [ ]:
shows_top10.plot(kind='bar', x='title', y='duration', legend=False)

plt.xlabel('')
plt.ylabel('# of Seasons')
plt.title("Netflix Shows with Most Seasons");

Looking closer at only the genres of the top 10 shows:

In [ ]:
#Obtain a list of all the genres
genres = []

for col in range(11, 32):
    genres.append(shows_genres.columns[col])

In [ ]:
#Get only the genres one-hot encoding for the top 10 shows
top10_genres = shows_top10[genres].reset_index(drop=True)

In [ ]:
top10_genres = top10_genres.transpose()

In [ ]:
#Get a sum of each genre in the top 10 shows
top10_genres['count'] = top10_genres.sum(axis=1)

#Obtain the genre as a percentage of total genres represented in the top 10 shows
top10_genres['percentage'] = top10_genres['count']/(top10_genres.to_numpy().sum())

#Sort by decreasing genre count
top10_genres = top10_genres.sort_values(['percentage', 'count'], ascending=False)

In [ ]:
top10_genres

In [ ]:
print("Out of the top 10 shows:\n")

for genre in top10_genres.index:
    print("{} is {:.2%}.".format(genre, top10_genres.loc[genre, 'percentage']))

It could be safe to say that the longest running shows are the most popular shows; after all, production companies wouldn't waste money to keep producing a failing or unpopular show.

From the top 10 longest/popular shows, we can see that `TV Comedies`, `Classic & Cult TV`, and `TV Dramas` are tied for the best, followed by `Crime TV Shows`. As such, we could possibly infer that producer should make a TV show of, or a blend of, those first 3 genres.

In [ ]:
shows_top10['rating'].value_counts()

By also looking at the ratings of those TV shows, the producer should most likely make a `TV-14` rated show in the category of `TV Comedy`, `Classic & Cult TV`, and/or `TV Dramas`. 

However, just because the top 10 popular shows were in particular genres, does not mean that said genres will be popular. These shows could be outliers, or exceptions.

In order to further investigate this, we can look at the TV shows' genres that have been added to Netflix each year.

In [ ]:
shows_genres_yearly = shows_genres.groupby('year_added')[genres].sum()
shows_genres_yearly

We can drop the `0` and `2020` rows since they don't mean much. `2020` data isn't reliable since it only contains one day of 2020, and `0` data is content that had no dates/were null.

In [ ]:
shows_genres_yearly.drop(index=[0, 2020], inplace=True)

In [ ]:
plt.figure(figsize=(16, 8))

sns.heatmap(shows_genres_yearly, fmt='g', annot=True, cmap='Blues')

plt.ylabel('Year')
plt.title('Heatmap of Netflix TV Show Genres added Annually');

From the above heatmap, let's first ignore the large section of international TV shows that get added since that 'genre' encompasses a whole umbrella of genres.

Without the international TV shows, we can see that most recently in 2019, `TV Dramas`, `TV Comedies`, `Crime TV Shows` were added the most, respectively. This trend is the same since 2017. 

`Classic & Cult TV` actually had the lowest content amount added to Netflix for the last 2 years. As such, it would not be a good idea for a producer to make a `Classic & Cult TV` show.

Aggregating both analyses together, we can conclude that a producer should make a TV Show with the `TV Dramas` and/or `TV Comedies` genres to have the best hopes of getting a TV Show added to Netflix (even better if it is an `International TV Show`).

In [ ]:
jovian.commit(project="zerotopandas-course-project")

## *Q2: When would be most preferable to release content?*

Now that we know what type of content is popular, it could be helpful to see when that content should be released. 

First, we can look at the difference between release dates and dates the content is added to Netflix. Additionally, we can look at which month has the least content added for the least competition. 

In [ ]:
netflix_data[netflix_data['year_added'].notna()==False]

Since there isn't that much content without release dates, I'm going to be dropping those for this analysis.

In [ ]:
#Drop the rows with NaT in date added
netflix_withdates = netflix_data.copy()
netflix_withdates = netflix_withdates.dropna(subset=['year_added'])

#Reformat the year_added column to int for easier comparison
netflix_withdates['year_added'] = netflix_withdates['year_added'].astype(int)

In [ ]:
#Calculate how long from content release did it get added to Netflix
netflix_withdates['release_diff'] = netflix_withdates['year_added'] - netflix_withdates['release_year']

In [ ]:
netflix_withdates['release_diff'].unique()

It seems odd that there are negative numbers, since it is highly unlikely to add content that's not been released, especially because it doesn't seem like Netflix does any pre-releases of content. As such, a closer look at this content is needed.

In [ ]:
netflix_withdates[netflix_withdates['release_diff']<0]

By Googling these titles, it is clear that at least some of their release dates are wrong. Since there isn't a lot of content with release dates prior to dates added to Netflix, they'll be dropped.

In [ ]:
#Remove the rows with release years that're before theirs year added to Netflix
netflix_withdates = netflix_withdates[netflix_withdates['release_diff']>=0]

In [ ]:
temp_df = netflix_withdates[['release_diff', 'type']].value_counts().reset_index()
temp_df = temp_df.rename({0: 'count'}, axis='columns')

#Plot the graph
ax = sns.lineplot(data=temp_df, x='release_diff', y='count', hue='type')
ax.set(xticks=range(0, 100, 10), xlabel='Year(s)', title='Time between Content Release and Netflix Addition')
plt.show()

A vast majority of the content is added one year from the release date, for both movies and TV shows.

Most people know studios often release their biggest content during the summer (or even winter) since that's when a lot of people have some free time, or are more likely to go out. Let's investigate whether this is the same strategy that Netflix employs when it comes to adding content to its catalog.

In [ ]:
#Create a list of months in the right order
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

#Get the content count by month/year
#Also remove the content from year 2020 since it's incomplete
temp2 = netflix_withdates.groupby('year_added')['month_added'].value_counts().drop(level='year_added', labels=2020).unstack().fillna(0)[month_order]

In [ ]:
plt.figure(figsize=(16, 8))

ax = sns.heatmap(temp2, fmt='g', annot=True, cmap='Blues')
ax.set(title='Heatmap of Netflix Content added Monthly', xlabel='', ylabel='')

plt.show()

From the above heatmap, it looks like more content gets added in the winter. This makes sense since Netflix is a streaming platform; instead of customers going outside to theatres to watch movies, they would be at home in order to watch the content. As such, winter months would be perfect since most people prefer to stay indoors during the colder months.

With this analysis, we can see that if producers would like their content to be added to Netflix, they'd most likely see this within a year of the release date in the winter months. As such, if the content were a movie, the theory is for maximum capitalization, is a theatre release in the summer (to attract the summer movie-goers) to be featured in Netflix that winter or the following one.

In [ ]:
jovian.commit(project="zerotopandas-course-project")

## *Q3: Who are the Netflix Top Directors?*

We can analyze who the top directors are, and then take a closer look at their content.

In [ ]:
netflix_directors = netflix_data[netflix_data['director'].notna()]

#Turn the genres of listed_in into a list to enable one hot encoding
netflix_directors['director'] = netflix_directors['director'].str.split(', ')

In [ ]:
netflix_directors['director']

In [ ]:
#Iterate through the list of genres and place a 1 in the corresponding column for every row in the dataframe
for index, row in netflix_directors.iterrows():
    for director in row['director']:
        netflix_directors.loc[index, director] = 1

In [ ]:
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
netflix_directors = netflix_directors.fillna(0)

In [ ]:
#Checking for any director duplicates
if (netflix_directors.columns.duplicated(keep=False).any() ==True):
    print('there are duplicates')
else: print('nope')

In [ ]:
directors_df = netflix_directors.iloc[:,11:].transpose()

In [ ]:
directors_df['count'] = directors_df.sum(axis=1)
directors_df = directors_df.sort_values('count', ascending=False)

In [ ]:
directors_df.head(10)

In [ ]:
print("List of top 15 directors with the most content on Netflix:\n")

for name in range(15):
    print("The #{} director with the most content is {}, with {} works."
         .format(name+1, directors_df.index[name], directors_df.iloc[name, 4265]))

Taking a closer look at the top directors:

In [ ]:
netflix_directors[netflix_directors['Jan Suter']==1]

In [ ]:
netflix_directors[netflix_directors['Johnnie To']==1]

Out of the top 10 directors in Netflix content, the top 5 directed Stand-Up Comedy content.

The other five are well-known directors.

In [ ]:
jovian.commit(project="zerotopandas-course-project")

## *Q4: Who are the major studios with content on Netflix, and how do they compare to each other?*

For this question, I'll work with an additional [dataset from Kaggle](https://www.kaggle.com/yazeidalqahtani/rotten-tomato-movie-reviwe): data from the popular movie review website Rotten Tomatoes.

First, I need to obtain the dataset in a usable format:

In [ ]:
#Path of the file to read
rt_filepath = "/kaggle/input/rotten-tomato-movie-reviwe/rotten tomato movie reviwe.csv"

#Read the file into a dataframe
rt_raw = pd.read_csv(rt_filepath)

I'll do some quick cleaning and parsing as well.

In [ ]:
#Changing the column name to match the movie title column of my first dataset
#Also to make some of them shorter
rt_raw = rt_raw.rename(columns={'Name': 'title', 'TOMATOMETER score': 't_score', 'TOMATOMETER Count': 't_count', 'AUDIENCE score': 'a_score', 'AUDIENCE count':'a_count'})

#Making sure the format of the titles are strings for easy comparisons
rt_raw['title'] = rt_raw['title'].astype(str)

#Reformatting AUDIENCE count as ints for easy comparisons
rt_raw['a_count'] = rt_raw.a_count.astype(str).str.strip().str.replace(',','').astype(int)

#Reformatting Studio names
rt_raw['Studio'] = rt_raw['Studio'].str.strip()

In [ ]:
rt_raw.info()

In [ ]:
temp_movie = movie_data.set_index(keys='title')
temp_rt = rt_raw.set_index(keys='title')

Now, I'll merge the two datasets together so that I can have a wider range of information. However, since I'll be merging the two datasets together, I'll only have "complete" information (i.e., relevant information from both datasets) for movies in both. 

As such, I can expect the volume of content to drastically decrease. This is especially so when I haven't thoroughly looked at the titles of each content; only exact matches as-is will be left.

In [ ]:
joint_df = pd.merge(temp_movie, temp_rt, how='inner', on='title')

In [ ]:
print("Old dataset volume: {} vs.\nNew dataset volume: {}."
      .format(movie_data.shape, joint_df.shape))

We can see that even before any clean-up, the data has dramatically decreased to roughly 10% of its original size, from `4265` datapoints to just `436`.

In [ ]:
joint_df.head()

Clean-up of the merged data:

In [ ]:
#Dropping redundant and irrelevant columns
joint_df = joint_df.drop(columns=['type', 'description', 'Rating', 'Directed By', 'Runtime'])

#Removing the content with less than 100 ratings since those wouldn't be too accurate
joint_df = joint_df[(joint_df.t_count >= 100) | (joint_df.a_count >= 100)]

In [ ]:
joint_df.head()

In [ ]:
#one hot encoding for studios
studio_df = joint_df[['Studio']]

#Iterate through the studio names to add a 1 to each corresponding studio column
for index, row in studio_df.iterrows():
    studio_df.loc[index, row] = 1
        
#Filling in the NaN values with 0
studio_df = studio_df.fillna(0)

#Drop the Studio column since it's unneeded
studio_df = studio_df.drop(columns='Studio')

In [ ]:
studio_df = studio_df.transpose()

In [ ]:
studio_df['count'] = studio_df.sum(axis=1)
studio_df = studio_df.sort_values(by='count', ascending=False)

In [ ]:
print("The studios with the most content on Netflix: \n")

for rank in range(10):
    print("#{} is {}, with {} works ({:.2%})."
         .format(rank+1, studio_df.index[rank], int(studio_df.iloc[rank, 374]), (studio_df.iloc[rank, 374]/studio_df.shape[1])))

It's no surprise that from Netflix contains the most content from its own production. However, this analysis is to be taken with a big grain of salt since roughly 90% of the original data was removed.

In [ ]:
jovian.commit(project="zerotopandas-course-project")

# Inferences and Conclusions

Below is a summary of the above analysis:
* Netflix's content is roughly 2/3 movies and 1/3 TV shows
* A majority of Netflix TV shows are only one season-long
* A majority of Netflix movies are roughly 100 minutes
* There isn't really any trend nor stability in the growth of Netflix's content, but there has been growth every year
* The most popular Netflix TV Shows are of TV Comedy/TV Drama/Classic & Cult TV genres with TV-14 ratings
* If a producer would like to create a TV show to be added to Netflix, it should be in the TV Comedy/TV Drama/International Show genre since those shows are being added the most to Netflix in the recent years
* Netflix content is usually added within the first year of release, in the winter months
* The directors with the most content on Netflix surprisingly are Comedy Stand-Up directors
* Unsurprisingly, Netflix seems to have a lot of its own produced content

# Future Work

Since I only added the Rotten Tomatoes data at the end and didn't get to explore it too much, it would be good to analyze that data in conjunction with the Netflix data or just as a standalone.

For example, how many of the popular movies from Rotten Tomatoes is on Netflix, or how Netflix movies compare to traditional studios' in terms of ratings (both critics and audience's).

In [ ]:
jovian.commit(project="zerotopandas-course-project")